In [1]:
import tensorflow as tf
from keras.models import load_model
import pickle
import numpy as np
import pandas as pd

In [2]:
# load the trained model, and the encoder pickle file (like one hot encoder pickle , scalar pickle etc)

model = load_model('ann_model.h5')

# load the encoder and scalar

with open('onehot_encoder_geography.pkl','rb') as file:
    onehot_encoder_geography=pickle.load(file)

with open('scalar.pkl','rb') as file:
    scalar = pickle.load(file)

In [3]:
# Example input data

input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [4]:
input_data_df = pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [5]:
geo_encoded = onehot_encoder_geography.transform([[input_data['Geography']]]).toarray()

c:\Users\ASUS\Desktop\genAI\ANN Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [6]:
geo_encoded

array([[1., 0., 0.]])

In [7]:
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder_geography.get_feature_names_out(['Geography']))

In [8]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [9]:
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

In [10]:
input_data_df['Gender'] = label_encoder_gender.transform([input_data['Gender']])

In [11]:
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [12]:
input_data_df = pd.concat([input_data_df.drop(['Geography'],axis=1),geo_encoded_df],axis=1)

In [13]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
# now one transformation is left i.e. scaling the input data

scaled_input_df = scalar.transform(input_data_df)


In [15]:
scaled_input_df

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
prediction = model.predict(scaled_input_df)

1/1 [==============================] - 0s 369ms/step


In [17]:
prediction

array([[0.00016691]], dtype=float32)